# **Time Series Regression Analysis**

## **<span style="color: blue;">Business Understanding</span>**

The main objective is to build a model that accurately predicts unit sales for thousands of products across different Favorita store locations. This will help ensure optimal inventory management, reduce waste, and improve operational efficiency.

We have access to several datasets:

1. Training data: Contains dates, store and product information, promotions, and sales figures.
2. Store metadata: Includes cluster, type, city, and state information.
3. Oil price data: Daily oil prices for the relevant time period.
4. Transactions data: Daily transaction counts for each store.

Key features include:

1. store_nbr: Identifies the store location
2. family: Product category
3. sales: Total sales for a product family at a specific store on a given date
4. onpromotion: Number of items in a product family on promotion

### **Hypothesis**

**Null Hypothesis (H0):**

Promotional activities (as measured by the 'onpromotion' variable) have no significant effect on the daily sales of products across Favorita stores.

Mathematically, we can express this as:
        
H0: β = 0

Where β is the coefficient of the 'onpromotion' variable in our predictive model.

**Alternative Hypothesis (H1):**

Promotional activities have a significant effect on the daily sales of products across Favorita stores.

Mathematically, we can express this as:

H1: β ≠ 0

### **Analytical Questions**

1. Is the train dataset complete (has all the required dates)?
2. Which dates have the lowest and highest sales for each year (excluding days the store was closed)?
3. Compare the sales for each month across the years and determine which month of which year had the highest sales.
4. Did the earthquake impact sales?
5. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
6. Are sales affected by promotions, oil prices and holidays?
7. What analysis can we get from the date and its extractable features?
8. Which product family and stores did the promotions affect.
9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
10. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.

## **<span style="color: blue;">Data Understanding</span>**

### **<span style="color: skyblue;">Importation</span>**

**Importation of all necessary packages**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Dataset Importation & Loading**

In [2]:
# Connect to the Database Using "pyodbc"
import pyodbc
print("pyodbc is installed and imported successfully")

import pyodbc
from dotenv import dotenv_values    #import the dotenv_values function from the dotenv package
import pandas as pd
import warnings 

warnings.filterwarnings('ignore')

# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials set in the '.env' file
server = environment_variables.get("SERVER_NAME")
database = environment_variables.get("DATABASE_NAME")
login = environment_variables.get("LOGIN")
password = environment_variables.get("PASSWORD")

pyodbc is installed and imported successfully


In [3]:
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={login};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"


In [4]:
# connect to the Database
connection = pyodbc.connect(connection_string)
print("Connection successful")

Connection successful


In [5]:
# Query the Sql Database
query = "SELECT * FROM dbo.oil"

# Execute the query and read the results into a DataFrame
oil = pd.read_sql(query, connection)

In [6]:
# Query the Sql Database
query = "SELECT * FROM dbo.holidays_events"

# Execute the query and read the results into a DataFrame
holidays_events = pd.read_sql(query, connection)

In [7]:
# Query the Sql Database
query = "SELECT * FROM dbo.stores"

# Execute the query and read the results into a DataFrame
stores = pd.read_sql(query, connection)

In [8]:
sample_submission = pd.read_csv('../Data/sample_submission.csv')

In [9]:
test = pd.read_csv('../Data/test.csv')

In [10]:
train = pd.read_csv('../Data/train.csv')

In [11]:
transactions = pd.read_csv('../Data/transactions.csv')

### **<span style="color: skyblue;">Exploratory Data Analysis</span>**

#### train data

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [22]:
# Convert date to datetime format

train['date'] = pd.to_datetime(train['date'])

In [24]:
# Create additional time-based features

train['day_of_week'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['weekend'] = train['day_of_week'].isin([5, 6]).astype(int)


In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
 6   day_of_week  int32         
 7   month        int32         
 8   year         int32         
 9   weekend      int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(4), object(1)
memory usage: 194.6+ MB


In [30]:
train.describe().T

,count,mean,min,25%,50%,75%,max,std
id,3000888.0,1500443.5,0.0,750221.75,1500443.5,2250665.25,3000887.0,866281.891642
date,3000888,2015-04-24 08:27:04.703088384,2013-01-01 00:00:00,2014-02-26 18:00:00,2015-04-24 12:00:00,2016-06-19 06:00:00,2017-08-15 00:00:00,NaN
store_nbr,3000888.0,27.5,1.0,14.0,27.5,41.0,54.0,15.585787
sales,3000888.0,357.775749,0.0,0.0,11.0,195.84725,124717.0,1101.997721
onpromotion,3000888.0,2.60277,0.0,0.0,0.0,0.0,741.0,12.218882
day_of_week,3000888.0,2.997031,0.0,1.0,3.0,5.0,6.0,2.00074
month,3000888.0,6.207838,1.0,3.0,6.0,9.0,12.0,3.385668
year,3000888.0,2014.837886,2013.0,2014.0,2015.0,2016.0,2017.0,1.345518
weekend,3000888.0,0.285629,0.0,0.0,0.0,1.0,1.0,0.451714


In [34]:
train.isnull().sum().sum()

np.int64(0)

In [35]:
train.duplicated().sum().sum()

np.int64(0)

#### test data

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [31]:
# Convert date to datetime format

test['date'] = pd.to_datetime(test['date'])

In [32]:
# Create additional time-based features

test['day_of_week'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['year'] = test['date'].dt.year
test['weekend'] = test['day_of_week'].isin([5, 6]).astype(int)


In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
 5   day_of_week  28512 non-null  int32         
 6   month        28512 non-null  int32         
 7   year         28512 non-null  int32         
 8   weekend      28512 non-null  int64         
dtypes: datetime64[ns](1), int32(3), int64(4), object(1)
memory usage: 1.6+ MB


In [36]:
# checking for missing values

test.isnull().sum().sum()

np.int64(0)

In [37]:
# Check for dublicates

test.duplicated().sum().sum()

np.int64(0)

In [38]:
test.describe().T

,count,mean,min,25%,50%,75%,max,std
id,28512.0,3015143.5,3000888.0,3008015.75,3015143.5,3022271.25,3029399.0,8230.849774
date,28512,2017-08-23 12:00:00,2017-08-16 00:00:00,2017-08-19 18:00:00,2017-08-23 12:00:00,2017-08-27 06:00:00,2017-08-31 00:00:00,NaN
store_nbr,28512.0,27.5,1.0,14.0,27.5,41.0,54.0,15.586057
onpromotion,28512.0,6.965383,0.0,0.0,0.0,6.0,646.0,20.683952
day_of_week,28512.0,2.9375,0.0,1.75,3.0,4.25,6.0,1.886457
month,28512.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0
year,28512.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,0.0
weekend,28512.0,0.25,0.0,0.0,0.0,0.25,1.0,0.43302
